# Прогнозирование стоимости автомобиля

### Цель проекта
Подобрать выбрать оптимальный тип и параметры модели для определения цены автомобиля по характеристикам.

### Задачи:
1. Создать тренировочный датасет, используя объявления с сайта auto.ru;
2. Обработать тренировочный и тестовый датасеты;
3. Поработать с дубликатами, пропусками;
4. Провести EDA;
5. Сделать Feature Engineering;
6. Перебрать модели обучения из разных библиотек и выбрать лучшую;
7. Провести подбор оптимальных параметров моделей.

### 1. Импортируем библиотеки необходимые для обработки данных

In [1]:
import numpy as np 
import pandas as pd 
import pandas_profiling
import warnings
warnings.simplefilter('ignore')
import sys
import time
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold
from tqdm.notebook import tqdm
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.base import clone
import xgboost as xgb

Зададим параметры отображения dataframe без ограничения количества столбцов и 30 строк по умолчанию:

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 30

### 2.Проверим версию пакетов

In [3]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [4]:
!pip freeze > requirements.txt

### 3. Функции для обработки данных

Фиксируем RANDOM_SEED:

In [5]:
RANDOM_SEED = 42

Запишем функцию для вычисления итоговой метрики:

In [6]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

Подсчет количества выбросов:

In [7]:
def data_outliers(column):  
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    min_meaning = Q1 - 1.5 * IQR
    max_meaning = Q3 + 1.5 * IQR
    return (column < min_meaning).sum() + (column > max_meaning).sum(), min_meaning, max_meaning

Построение boxplot:

In [8]:
def get_boxplot(df, col):
    fig, axes = plt.subplots(figsize = (14, 4))
    sns.boxplot(x='price', y=col, data=df, ax=axes)
    axes.set_title('Boxplot for ' + col)
    plt.show()

### 4.Код парсинга

Код прасинга получился очень затратным по времени, поэтому было принято решение о паралельной разработке.

Паралельно парсилась база данных и проходил обработку начальный датасет.

Прилагаю ссылку Parcing_code на github:

[Parcing_code.ipynb](https://github.com/Kondrashovko/skillfactory_rds/blob/main/module_6/Parcing_code.ipynb)

### 5.Импортируем данные необходимые для построения модели

Пропишем путь для датасетов:

In [9]:
path = '../input/sf-dst-car-price-prediction/'
#'D:/Python/files/ML/Itog_7/' 

Подгрузим test, осмотрим данные и опишем параметры:

In [10]:
test = pd.read_csv(path+'test.csv')

In [11]:
test.info()

In [12]:
test.sample(5)

В наборе данных следующие признаки:
* bodyType - тип кузова;
* brand - марка автомобиля;
* car_url - url страницы объявления;
* color - цвет автомобиля;
* complectation_dict - комплектация автомобиля;
* description - описание продавца;
* engineDisplacement - объём двигателя;
* enginePower - мощность двигателя;
* equipment_dict - дополнительные модули, установленные в автомобиль;
* fuelType - тип топлива;
* image - ссылка на картинку;
* mileage - пробег;
* modelDate - дата релиза модели;
* model_info - информация о модели автомобиля;
* model_name - модель автомобиля;
* name - имя, введенное пользователем;
* numberOfDoors - количество дверей;
* parsing_unixtime - время парсинга данных;
* priceCurrency - валюта указанной цены;
* production_date - дата производства автомобиля;
* sell_id - уникальный индетификатор продавца;
* super_gen - дополнительные параметры автомобиля;
* vehicleConfiguration - конфигурация транспортного средства (ТС);
* vehicleTransmission - тип коробки передач;
* vendor - страна производитель
* Владельцы - количество владельцев;
* Владение - время владения автомобилем;
* ПТС - паспорт ТС;
* Привод - тип привода;
* Руль - сторона руля;
* Состояние - состояние автомобиля;
* Таможня - этап растаможки;

Признаки могут оказаться малополезными и от их использования можно отказаться. 

Будем приводить параметры обучающего датасета к этим параметрам.

Также если в обучающем датасете не будет этих параметров, то либо придется парсить заново, либо удалить эти параметры.

Датасет для обучения модели:

In [13]:
train = pd.read_csv('../input/car-list-final/cars_list_final.csv') 
#train = pd.read_csv('../input/car-list/cars_list.csv') 

In [14]:
train.info()

In [15]:
train.sample(5)

В присутсвуют новые параметры. Описывать их нет смысла - таких параметров нет в тестовом датасете, и они подлежат удалению.

Также, некоторые параметры отличаются по своему содержанию и названию столбцов. Исправим все и приведем к общему виду при первичной обработке данных.

Загрузим sample_submission:

In [16]:
sample_submission = pd.read_csv(path+'sample_submission.csv')

### 6.Preprocessing

Большинство параметров представленных в тестовой выборке присутствуют и в обучающей выборке, только они имеют другие параметры. 

Обработаем обучающий датасет и приведем данные к общему знаменателю.

#### 6.1. engineDisplacement и fuelType

##### 6.1.a. Топливные автомобили

Для начала обработаем автомобили на топливе, электроавтомобили обработаем ниже:

Объем двигателя в обучающей модели можно извлечь из параметра 'Двигатель':

In [17]:
train.engineDisplacement = train.Двигатель.apply(lambda x: x[:4])

Также в тестовой выборке оставим только сам объем двигателя:

In [18]:
test.engineDisplacement = test.engineDisplacement.apply(lambda x: x[:4])

Установить какой тип топлива используется в автомобиле мы не можем, поэтому применяем топлива для всех автомобилей:

In [19]:
train['fuelType'] = 'Топливо'

Также заменим на топливо и в тестовом датасете:

In [20]:
fuel_dict = {'бензин':'Топливо', 
             'дизель':'Топливо', 
             'гибрид':'Гибрид', 
             'электро':'Электро', 
             'газ':'Топливо'}
test['fuelType'] = test['fuelType'].replace(to_replace=fuel_dict)

Мощность двигателя в обучающей выборке можно извлечь из параметра 'Двигатель':

In [21]:
train.enginePower = train.Двигатель.apply(lambda x: x[9:12])
train.enginePower = train.enginePower.str.replace('/ ', '0')
train.enginePower = train.enginePower.apply(lambda x: int(x))

В тестовой выборке только мощность:

In [22]:
test.enginePower = test.enginePower.apply(lambda x: int(x[:3]))

##### 6.1.b. Электроавтомобили

Электроавтомобилей не так много и из обработка требует немного другого подхода. Поэтому переберем все данные и извлечем данные двигателя для обучающих и тестовых данных:

In [23]:
for i in range(len(train)):
    if 'Электро' in train.Двигатель[i]:
        train.enginePower[i] = 0
        train['fuelType'][i] = 'Электро'

In [24]:
for i in range(len(test)):
    if 'Electro' in test.name[i]:        
        test.engineDisplacement[i] = 0

Рабочий объем элетродвигателя связан с топливом, поэтому параметр 'engineDisplacement' запишем равный 0.

##### 6.1.c. Приведение к численным параметрам

Запишем новые данные в датасет:

In [25]:
train['enginePower'] = train.enginePower
test['enginePower'] = test.enginePower
train['engineDisplacement'] = train.engineDisplacement
test['engineDisplacement'] = test.engineDisplacement

Переведем engineDisplacement в float64:

In [26]:
train['engineDisplacement'] = train['engineDisplacement'].astype('float64')
test['engineDisplacement'] = test['engineDisplacement'].astype('float64')

#### 6.2. Пробег

Пробег имеет лишние пробелы и подпись 'км'. избавимся от них. 

In [27]:
train['Пробег'] = train['Пробег'].str.replace(' ', '')
train['Пробег'] = train['Пробег'].apply(lambda x: int(x[:-2]))

Позже переименуем колонку в 'mileage' как в тестовом датасете. 

#### 6.3. name

Имя модели параметр текстовый и из него мало чего получить при обучении модели. Поэтому удалим брэнд и  все вариации описания модели:

In [28]:
train['name'] = train.name.apply(lambda x: x.split(' ')[1].upper())

Позже переименуем колонку в 'model_name' как в тестовом датасете. 

#### 6.4. numberOfDoors

К сожалению в обучающей выборке не оказалось данных о количестве дверей. Их можно восстановить по типу кузова. Введем словарь для количества дверей:

In [29]:
doors_dict = {'лифтбек':5, 
              'внедорожник 5 дв.':5, 
              'седан':4, 
              'хэтчбек 5 дв.':5,
              'универсал 5 дв.':5,
              'минивэн':5,
              'купе':5,
              'хэтчбек 3 дв.':3,
              'компактвэн':5,
              'внедорожник 3 дв.':3,
              'фургон':5,
              'пикап двойная кабина':2,
              'купе-хардтоп':2,
              'кабриолет':4,
              'родстер':2,
              'фастбек':2,
              'пикап одинарная кабина':2,
              'пикап полуторная кабина':2,
              'седан 2 дв.':2,
              'тарга':2,
              'внедорожник открытый':0,
              'седан-хардтоп':4,
              'лимузин':4,
              'микровэн':5,
              'хэтчбек 4 дв.':4,
             }

Также в обучающей выборке перед некоторыми параметрами спарсился пробел. Исправим это для кузова:

In [30]:
train['Кузов'] = train['Кузов'].apply(lambda x: x[1:])

Восстановим количество дверей:

In [31]:
train['numberOfDoors']= train['Кузов'].map(doors_dict)

##### 6.6. brand

В обучающей выборке много брендов. Тогда как в тестовой их не так много:

In [32]:
Car_type = list(test['brand'].unique())
Car_type

Поскольку 'brand' - категориальный параметр, в котором нужно будет выбрать самые популярные марки, то лучше заранее оставить важные марки:

In [33]:
train['brand'] = train['brand'].apply(lambda x: x if x in Car_type else 'other')

#### 6.6. vendor

Восстановим вендора через словарь:

In [34]:
vendor_dict = {'BMW' : 'EUROPEAN', 
               'MERCEDES' : 'EUROPEAN', 
               'VOLKSWAGEN' : 'EUROPEAN', 
               'TOYOTA' : 'JAPANESE', 
               'NISSAN' : 'JAPANESE',
               'AUDI' : 'EUROPEAN',
               'SKODA' : 'EUROPEAN', 
               'MITSUBISHI' : 'JAPANESE',
               'INFINITI' : 'JAPANESE', 
               'HONDA' : 'JAPANESE',
               'VOLVO' : 'EUROPEAN',
               'LEXUS' : 'JAPANESE',
               'other': 'other'
              }

In [35]:
train['vendor'] = train['brand'].replace(to_replace=vendor_dict)

#### 6.7. price

Цены для тестовой выборки неизвестны и поэтому равны 0:

In [36]:
test['price'] = 0

#### 6.8. Обработка ошибок парсинга

Также в обучающей выборке перед некоторыми параметрами спарсился пробел. Исправим это для всех замеченных параметров:

In [37]:
train['Коробка'] = train['Коробка'].apply(lambda x: x[1:])
train['Привод'] = train['Привод'].apply(lambda x: x[1:])
train['Руль'] = train['Руль'].apply(lambda x: x[1:])
train['Состояние'] = train['Состояние'].apply(lambda x: x[1:])
train['Таможня'] = train['Таможня'].apply(lambda x: x[1:])

#### 6.9. Переименование колонок

Переименуем колонки обучающей выборки в соответствие с тестовой выборкой:

In [38]:
train.rename(columns = {'Кузов' : 'bodyType', 
                        'url':'car_url', 
                        'name' : 'model_name',
                        'Пробег':'mileage', 
                        'Коробка':'vehicleTransmission'}, inplace = True) 

#### 6.10. Очистка данных от дубликатов

У каждого автомобиля своя уникальная ссылка. По ней и будем дубликаты:

In [39]:
train = train.drop_duplicates(subset=['car_url'])

Проверим количество дубликатов:

In [40]:
train['car_url'].duplicated().sum()

#### 6.11. Сохранение данных перед очисткой

В нашем обучение не пригодится 'parsing_unixtime'. Однако его можно обработать для других целей: 

In [41]:
test['parsing_unixtime'].median()

In [42]:
parcing_date = datetime.fromtimestamp(test['parsing_unixtime'].median()).strftime("%d/%m/%Y")
parcing_date

Средняя дата парсинга - 21 октября 2020 года. 

Данные же, используемые в обучающем датасете, собраны в конце января 2022.

Используем это в дальнейшем, чтобы учесть инфляцию при обучении.

#### 6.12. Очистка данных

Удалим лишние данные:

In [43]:
train.drop(['@context', 
            '@type', 
            'Двигатель',
            'год выпуска', 
            'offers',
            'Цвет',
            'Комплектация',
            'Налог',
            'Гарантия',
            'VIN',            
            'Госномер',
            'availability',
            'Обмен',            
            'Запас хода',
            'Кузов №',
            ], axis=1, inplace=True)

In [44]:
test.drop(['complectation_dict', 
           'equipment_dict', 
           'modelDate',
           'model_info', 
           'name', 
           'vehicleConfiguration',
           'parsing_unixtime',
           'super_gen'
          ], axis=1, inplace=True)

Удалим строки, где отсутствует цена

In [45]:
train = train.dropna(axis=0, subset=['price'])

Прежде чем приступать к обработке данных посмотрим выбросы в целевой переменной:

In [46]:
data_outliers(train['price'])

In [47]:
train['price'].min()

Отрицательной цены нет, значит выбросов по минимальной цене нет

5583 выбросов с ценой выше 4 миллионов рублей. Посмотрим максимальное значение:

In [48]:
train[train['price'] == train['price'].max()]

Явный выброс:

In [49]:
train = train.loc[train['price'] != train['price'].max()]

In [50]:
train = train.loc[train['price'] < 20000000]

Посмотрим цены выше 10 миллионов рублей и год их выпуска:

In [51]:
train[train['price'] > 10000000].sample(10)

In [52]:
train[train['price'] > 10000000].productionDate.value_counts()

Новые и раритетные автомобили. Вроде все нормально.

Посмотрим цены выше 20 миллионов рублей:

In [53]:
train[train['price'] > 20000000]

In [54]:
train[train['price'] > 20000000].productionDate.value_counts()

Это уже выбросы. Удаляем эти данные из датасета:

In [55]:
train = train.loc[train['price'] < 20000000]

### 7.Объединяем данные

Для корректной обработки признаков объединяем трейн и тест в один датасет и ставим метку обучающей части:

In [56]:
train['sample'] = 1 
test['sample'] = 0 

Объединяем данные:

In [57]:
data = test.append(train, sort=False).reset_index(drop=True) # объединяем

Осматриваем результат:

In [58]:
data.sample(5)

Очистим данные от дубликатов:

In [59]:
data = data.drop_duplicates(subset=['car_url'])

### 8. Обработка NAN 

Осмотрим данные на пропуски:

In [60]:
data.isna().sum()

Обработаем и, по возможности, восполним пропущенные параметры:

#### 8.1. sell_id

'sell_id' - уникальный параметр. Его можно удалить:

In [61]:
data = data.drop(columns = ['sell_id'], axis=1)

#### 8.2. Владельцы

Заполним пропуски самым популярным значением:

In [62]:
data['Владельцы'] = data['Владельцы'].fillna('3')

Удалим пробелы из данных:

In [63]:
data['Владельцы'] = data.Владельцы.str.replace(' ', '')

Оставим только количество, как численный параметр:

In [64]:
data['Владельцы'] = data['Владельцы'].apply(lambda x: int(x[0]))

Осмотрим получившийся параметр:

In [65]:
data['Владельцы'].value_counts()

#### 8.3. Владение

Если продавец не указал время владения автомобилем - это салон или перекупщик. Сделаем из этого параметра бинарный признак:

In [66]:
data['Владение'] = data['Владение'].isna().astype(np.int8)

#### 8.4. ПТС

Осмотрим строки, где не указали ПТС:

In [67]:
data[data['ПТС'].isna()]

Поверим людям и заполним пропуски популярным значением 'Оригинал'

In [68]:
data['ПТС'] = data['ПТС'].fillna('Оригинал')

Удалим пробелы из данных:

In [69]:
data['ПТС'] = data.ПТС.str.replace(' ', '')

Осмотрим результаты:

In [70]:
data['ПТС'].value_counts()

#### 8.5. Статус

Данных очень мало и их не восстановить. Удаляем параметр:

In [71]:
data = data.drop(columns = ['Статус'], axis=1)

### 9. Aнализ, распределение признаков и Feature Engineering

Посмотрим, что pandas_profiling расскажет о наших данных:

In [72]:
pandas_profiling.ProfileReport(data)

Ознакомившись с результатами, можно закоментить эту функцию для большей производительности.

Предлагается удалить несколько параметров, которые уникальны или не имеют отличий. Осмотрим только параметр 'Состояние':

In [73]:
test['Состояние'].value_counts()

В тестовой выборке битых машин нет. Осмотрим битые машины подробнее:

In [74]:
data[data['Состояние'] == 'Битый / не на ходу']

Эти данные лучше удалить - цена существенно занижена:

In [75]:
data = data.loc[data['Состояние'] != 'Битый / не на ходу']

Теперь удаляем столбецы:
- 'Состояние';
- 'car_url', 'sell_id' - параметры не несущие данных об автомобиле;
- 'priceCurrency', 'Таможня' - столбцы с единственным значением.

In [76]:
data = data.drop(columns = ['car_url',
                            'priceCurrency',
                            'Таможня',
                            'Состояние'], axis=1)

Также рассмотрим параметры и распределим их по типу: 
* бинарные;
* категориальные;
* числовые.

In [77]:
bin_cols = [] 
cat_cols = [] 
num_cols = []

#### 9.1. bodyType

Осмотрим уникальные параметры bodyType:

In [78]:
data.bodyType.value_counts()

Очистим данные и оставим первое слово из описания:

In [79]:
data['bodyType'] = data.bodyType.apply(lambda x: x.split(' ')[0].lower())
data.bodyType.value_counts()

Выделим самые популярные типы кузова:

In [80]:
popular_bodyType = data.bodyType.value_counts()[:9]
popular_bodyType

Заменим наименее популярные типы кузова описание 'другой':

In [81]:
data['bodyType'] = data['bodyType'].apply(lambda x: x if x in popular_bodyType else 'другой')

Осмотрим полученный результат в графической интерпретации:

In [82]:
get_boxplot(data, 'bodyType')

In [83]:
data.bodyType.value_counts().plot.barh() 

In [84]:
plt.figure(figsize=(8, 5))
g1 = sns.boxplot(y='price', x='bodyType', 
                  data=train, color='darkgreen')
g1.set_title("price of bodyType", fontsize=20)

g1.set_ylabel("price", fontsize=15)
g1.set_xticklabels(g1.get_xticklabels(),rotation=45)
plt.show()

Посмотрим на распределение зависимости цены от типа кузова:

In [85]:
plt.figure(figsize=(10, 15))
plt.scatter(np.log(data.price), data.bodyType)

Признак относится к категориальным:

In [86]:
cat_cols.append('bodyType')

#### 9.2. brand

Осмотрим уникальные параметры 'brand':

In [87]:
data.brand.value_counts()

Удаляем данные других брэндов. Они могут влиять на конечный результат:

In [88]:
data = data.loc[data['brand'] != 'other']

Осмотрим графически:

In [89]:
get_boxplot(data, 'brand')

In [90]:
data.brand.value_counts().plot.barh() 

Признак относится к категориальным:

In [91]:
cat_cols.append('brand')

#### 9.3. model_name

Для начала выровняем регистр и очистим данные 'model_name':

In [92]:
data['model_name'] = data.model_name.apply(lambda x: x.upper())

In [93]:
data['model_name'] = data.model_name.str.replace('-', '_')
data['model_name'] = data.model_name.str.replace('KLASSE', 'КЛАСС')
data['model_name'] = data.model_name.str.replace('_AMG', '')
data['model_name'] = data.model_name.str.replace('(', '')
data['model_name'] = data.model_name.str.replace(')', '')

Осмотрим уникальные параметры 'model_name':

In [94]:
data.model_name.value_counts()

Выделим 30 самых популярных моделей:

In [95]:
popular_name = data.model_name.value_counts()[:30]
popular_name

Также осмотрим редкость автомобиля. Если количество объявлений меньше 15, то скорее всего редкий и его цена существенно выше:

In [96]:
rare_car = data['model_name'].value_counts().to_frame()
rare_model = data.model_name.value_counts()[-len(rare_car[rare_car['model_name'] < 15]):]
rare_model

Создадим параметр 'rare' - редкость автомобиля:

In [97]:
data['rare'] = data['model_name'].apply(lambda x: 1 if x in rare_model else 0)

Заменим непопулярные автомобили параметром 'other':

In [98]:
data['model_name'] = data['model_name'].apply(lambda x: x if x in popular_name else 'other')

Осмотрим полученные данные:

In [99]:
get_boxplot(data, 'model_name')

- 'model_name' - категориальный признак;
- 'rare' - бинарный.

In [100]:
cat_cols.append('model_name')
bin_cols.append('rare')

#### 9.4. color

Осмотрим boxplot:

In [101]:
get_boxplot(data, 'color')

Категориальный признак

In [102]:
cat_cols.append('color')

#### 9.5. description

Сгенерируем из описания продавца новый числовой признак 'description_len' - длина описания:

In [103]:
data['description_len'] = data.description.apply(lambda x: len(str(x)))

Также осмотрим уникальность описания

In [104]:
simular_description = data['description'].value_counts().to_frame()
simular_description.head()

Некоторые салоны предлaгают к сотням машин одинаковое описание. 

Выделим уникальные объявления, а остальных отнесем к перекупам:

In [105]:
len_not_secondhand = len(simular_description[simular_description['description'] == 1])
not_secondhand = data.description.value_counts()[-len_not_secondhand:]
data['secondhand_dealer'] = data['description'].apply(lambda x: 0 if x in not_secondhand else 1)

In [106]:
data['secondhand_dealer'].value_counts()

Категории:
- 'description' - под удаление;
- 'description_len' - числовой тип;
- 'secondhand_dealer' -  бинарный параметр.

In [107]:
data = data.drop('description',axis=1)
num_cols.append('description_len')
bin_cols.append('secondhand_dealer')

#### 9.6. fuelType

Осмотрим boxplot категориального признака:

In [108]:
get_boxplot(data, 'fuelType')

In [109]:
cat_cols.append('fuelType')

#### 9.7. image

Осмотрим уникальность ссылки на картинку:

In [110]:
data.image.value_counts()[:10]

Ссылки на картинку повторяются десятки раз. 

Получим уникальные фотографии, по признаку 'get-verba' - это ссылка на получение картинки:

In [111]:
data['seller_photo'] = data['image'].apply(lambda x: 1 if 'get-verba' in x else 0)

In [112]:
data['seller_photo'].value_counts()

Категории:
- 'image' - под удаление;
- 'seller_photo' - бинарный признак.

In [113]:
data = data.drop('image',axis=1)
bin_cols.append('seller_photo')

#### 9.8. engineDisplacement

Разобьем по категориям. Вспомним, что объем электродвигателя равен 0:

In [114]:
data["engineDisplacement"] = data["engineDisplacement"].apply(lambda x: 6 if x > 6 else round(x))

In [115]:
sns.countplot(x = 'engineDisplacement', data = data)

In [116]:
cat_cols.append('engineDisplacement')

#### 9.9. enginePower

Осмотрим числовой признак:

In [117]:
data.enginePower.hist().barh

Осмотрим квартили и обработаем выбросы:

In [118]:
Power_outliers=data_outliers(data['enginePower'])
Power_outliers

In [119]:
data['enginePower'].min()

Всего 3568 выбросов. 

Проверили, что минимальная мощность двигателя больше 0. Все выбросы - очень мощные машины.

Введем бинарный признак 'силовых' машин, у кого мощность больше верхнего квартиля - запишем в категорию 'muscle_car':

In [120]:
muscle = Power_outliers[2]
data["muscle_car"] = data["enginePower"].apply(lambda x: 1 if x > muscle else 0)

Категории:
- 'enginePower' - числовой тип;
- 'muscle_car' -  бинарный параметр.

In [121]:
num_cols.append('enginePower')
bin_cols.append('muscle_car')

#### 9.10. mileage

Осмотрим числовой признак:

In [122]:
data['mileage'].hist(figsize=(5,5), bins=50)

Осмотрим квартили и обработаем выбросы:

In [123]:
mileage_outliers=data_outliers(data['mileage'])
mileage_outliers

In [124]:
data['mileage'].min()

Всего 1706 выбросов. 

Минимальный пробег 1, хотя при парсинге было явно указано "used". Больше 0, поэтому его выбросом не считаем.

Все выбросы - очень большой пробег.

Введем категориальный признак 'mileage_cat', чтобы разбить пробег автомобилей по категориям

In [125]:
data["mileage_cat"] = data["mileage"].apply(lambda x: 9 if x//50000 > 9 else x//50000)

In [126]:
data["mileage_cat"].value_counts()

Категории:
- 'mileage' - числовой;
- 'mileage_cat' -  категориальный.

In [127]:
num_cols.append('mileage')
cat_cols.append('mileage_cat')

#### 9.11. numberOfDoors

Осмотрим категориальный признак:

In [128]:
sns.countplot(x = 'numberOfDoors', data = data)

In [129]:
cat_cols.append('numberOfDoors')

#### 9.12. production_date

Осмотрим числовой признак:

In [130]:
sns.countplot(x = 'productionDate', data = data)

Осмотрим квартили и обработаем выбросы:

In [131]:
production_outliers=data_outliers(data['productionDate'])
production_outliers

In [132]:
data['productionDate'].max()

Всего 2579 выбросов. 

Самые новые автомобили 2022 года выпуска, хотя при парсинге было явно указано "used". 

Это возможно, и явно не выброс.

Все выбросы - раритетные автомобили, которым больше 30 лет.

Введем числовой признак возраста автомобиля 'age':

In [133]:
data['age'] = 2022 - data.productionDate

In [134]:
data_outliers(data['age'])

Выбросы нового числового признака сродни с родительским признаком, что неудивительно. Введем еще один категориальный признак - разобъем возраст по категориям:

In [135]:
data["category_age"] = data["age"].apply(lambda x: 3 if x//10 > 3 else x//10)

Все выбросы попали в 3 категорию, поэтому не будем вводить бинарный признак раритетного автомобиля.

Осмотрим новый признак:

In [136]:
sns.countplot(x = 'category_age', data = data)

Определим категории:
- 'productionDate' - под удаление, т.к. есть признак линейно заменяющий дату производства;
- 'age' - числовой;
- 'category_age' -  категориальный.

In [137]:
data = data.drop('productionDate',axis=1)
num_cols.append('age')
cat_cols.append('category_age')

#### 9.12. vehicleTransmission

Осмотрим категориальный признак:

In [138]:
sns.countplot(x = 'vehicleTransmission', data = data)

In [139]:
get_boxplot(data, 'vehicleTransmission')

In [140]:
cat_cols.append('vehicleTransmission')

#### 9.13. vendor

Осмотрим категориальный признак:

In [141]:
get_boxplot(data, 'vendor')

In [142]:
cat_cols.append('vendor')

#### 9.14. Владельцы

Осмотрим категориальный признак:

In [143]:
sns.countplot(x = 'Владельцы', data = data)

In [144]:
cat_cols.append('Владельцы')

#### 9.15. Владение

Осмотрим бинарный признак:

In [145]:
sns.countplot(x = 'Владение', data = data)

In [146]:
bin_cols.append('Владение')

#### 9.16. ПТС

Осмотрим признак:

In [147]:
get_boxplot(data, 'ПТС')

Преобразуем в бинарный признак:

In [148]:
data['ПТС'] = data['ПТС'].apply(lambda x: 1 if x == 'Оригинал' else 0).astype('int8')
bin_cols.append('ПТС')

#### 9.17. Руль

Осмотрим признак:

In [149]:
get_boxplot(data, 'Руль')

Преобразуем в бинарный признак:

In [150]:
data['Руль'] = data['Руль'].apply(lambda x: 1 if x == 'Левый' else 0).astype('int8')
bin_cols.append('Руль')

#### 9.18. Привод

Осмотрим категориальный признак:

In [151]:
get_boxplot(data, 'Привод')

In [152]:
cat_cols.append('Привод')

#### 9.19. price

Средняя дата парсинга, как упоминалось выше, -  21 октября 2020 года. 

In [153]:
parcing_date

Данные же, используемые в обучающем датасете, собраны в конце января 2022.

Мы должны обработать признак цены автомобиля с учетом инфляции

<img src="https://www.autostat.ru/application/includes/blocks/big_photo/images/original/f2a/8c2/366853b406d0c5a564298e0c31.jpg" width="90%" height = "60%" alt="">

Источник картинки [www.autostat.ru](https://www.autostat.ru/infographics/48987/)

С 2020 по 2022 год цены на автомобили сильно поднялись, особенно на популярные модели. Почитать об этом можно по ссылкам:

[Цены на автомобили с пробегом растут быстрее, чем на новые](https://mag.auto.ru/article/risepricescalltouchpres/)

[Средняя цена автомобилей в Москве в массовом сегменте](https://cenamashin.ru/statistika/moskva/avg_price?seg=1)

[Как менялись цены на автомобили с пробегом в России?](https://www.autostat.ru/infographics/48987/)

Будем использовать статистику с [cenamashin.ru](https://cenamashin.ru/statistika/moskva/avg_price?seg=1)

    Средняя цена трехлетнего автомобиля в октябре 2020:       738767

    Средняя цена автомобиля в октябре 2020:                   584605

    Средняя цена трехлетнего автомобиля сейчас:               1107344

    Средняя цена автомобиля сейчас:                           790063

Посчитаем инфляцию для двух параметров:

In [154]:
inflation_3year = 1107344/738767
display('Средняя инфляция трехлетнего автомобиля: ' + str(round(inflation_3year, 3)))
inflation_all = 790063/584605
display('Средняя инфляция цена автомобиля: ' + str(round(inflation_all, 3)))

Инфляция 50 % - это вполне вероятно, но слишком большая. 

35% - вроде правдивая инфляция, учитывая простой автопроизводств из-за дифицита полупроводников.

[Дефицит полупроводников останавливает мировой автопром](https://quote.rbc.ru/news/article/605de55e9a7947b757afd891)

[Кризис полупроводников в 2022 году: удорожание чипов, усугубление проблем автомобилестроения, игровой отрасли и медицины](https://habr.com/ru/company/selectel/blog/561986/)

[Почему дефицит микросхем – это миф, но автокомпаниям и дилерам он выгоден](https://5koleso.ru/blogi/pochemu-deficzit-mikroshem-eto-mif-no-avtokompaniyam-i-dileram-on-vygoden/)

[Глобальный дефицит полупроводников: идеальный шторм](https://econs.online/articles/ekonomika/globalnyy-defitsit-poluprovodnikov-idealnyy-shtorm/)

Cущественная инфляция на новые автомобили в 50% за полтора года - вполне реальная цифра.

Вслед за новыми автомобилями дорожают и автомобили с пробегом. В качестве коэффициента инфляции примем среднее между полученными коэффициентами инфляции.

In [155]:
inflation = (inflation_3year + inflation_all) / 2
inflation

Итоговая инфляция 42.5 процента. За полтора года с довольно насыщенными различными событиями вполне реальная инфляция.

Поделим целевой признак на полученный коэффициент инфляции:

In [156]:
data['price'] = data['price']/inflation

### 10. Обработка признаков и поиск корреляций

#### 10.1. Числовые признаки 

Применим стандартизацию к числовым признакам:

In [157]:
data[num_cols] = StandardScaler().fit_transform(data[num_cols].values)

In [158]:
data[num_cols]

#### 10.2. Бинарные признаки

Заменяем бинарные признаки 0 и 1 с помощью LabelEncoder:

In [159]:
label_encoder = LabelEncoder()
for column in bin_cols:
    data[column] = label_encoder.fit_transform(data[column])

In [160]:
data[bin_cols]

#### 10.3. Категориальные признаки

In [161]:
for column in cat_cols:
    data[column] = data[column].astype('category').cat.codes

#### 10.4. Поиск корреляций

In [162]:
corr = data.corr()
cmap = sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector='th',
                 props=[('font-size', '7pt')]),
            dict(selector='td',
                 props=[('padding', '0em 0em')]),
            dict(selector='th:hover',
                 props=[('font-size', '12pt')]),
            dict(selector='tr:hover td:hover',
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption('Hover to magify')\
    .set_precision(2)\
    .set_table_styles(magnify())

Высокий коэффициент корреляции между:
- пробегом и категорией пробега;
- корреляции между возрастом и категорией возраст;
- enginePower и muscle_car.

Все признаки конкурируют между собой, но у дочерних признаков уменьшается корреляция с другими признаками.

Хотя удаление каждого из параметров очень сильно снижает результат, поэтому enginePower останется в датасете.

enginePower

### 11. Train Split

По метке разделяем данный на обучающие и тестовые:

In [163]:
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample','price'], axis=1)

Определяем X и y:

In [164]:
X = train_data.drop(['price'], axis=1)
y = train_data['price']

Делаем Train Split:

In [165]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=RANDOM_SEED)

### 12. Model choice

Содзадим датасет для хранения предсказанной точности:

In [166]:
MAPE = []

#### 12.1. "Наивная" модель 

In [167]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

Находим median по экземплярам engineDisplacement в трейне и размечаем тест:

In [168]:
predict = X_test['engineDisplacement'].map(tmp_train.groupby('engineDisplacement')['price'].median())

Оцениваем точность:

In [169]:
naive_mape = mape(y_test, predict.values)
MAPE.append({'MAPE наивной модели': naive_mape})
print(f"Точность наивной модели по метрике MAPE: {(naive_mape)*100:0.2f}%")

#### 12.2. Простая модель линейной регрессии

In [170]:
linear_regr = LinearRegression().fit(X_train, np.log(y_train+1))
predict_test = np.exp(linear_regr.predict(X_test))

Оцениваем точность:

In [171]:
linear_mape = mape(y_test, predict_test)
MAPE.append({'MAPE линейной регрессии': linear_mape})
print(f"Точность модели линейной регрессии по метрике MAPE: {(linear_mape)*100:0.2f}%")

#### 12.3. CatBoost

#### 12.3.a. CatBoost without Log

In [172]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         )

model.save_model('catboost_single_model_baseline.model')

In [173]:
predict_CatBoost = model.predict(X_test)

Оцениваем точность:

In [174]:
CatBoost_mape = mape(y_test, predict_CatBoost)
MAPE.append({'MAPE CatBoost': CatBoost_mape})
print(f"Точность модели CatBoost по метрике MAPE: {(CatBoost_mape)*100:0.2f}%")

#### 12.3.b. CatBoost with Log Target

Попробуем взять таргет в логарифм - это позволит уменьшить влияние выбросов на обучение модели.    

In [175]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, np.log(y_train),
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         )

model.save_model('catboost_single_model_2_baseline.model')

In [176]:
predict_CatBoost_log = np.exp(model.predict(X_test))

Оцениваем точность:

In [177]:
CatBoost_mape_log = mape(y_test, predict_CatBoost_log)
MAPE.append({'MAPE CatBoost with log': CatBoost_mape_log})
print(f"Точность модели CatBoost с логарифмированием по метрике MAPE: {(CatBoost_mape_log)*100:0.2f}%")

#### 12.4. xgboost

In [178]:
xb = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.03, \
                      max_depth=12, alpha=1, n_jobs=-1, n_estimators=1000)
xb.fit(X_train, np.log(y_train+1))
VERSION = 7

In [179]:
predict_xgboost = np.exp(xb.predict(X_test))

Оцениваем точность:

In [180]:
xgboost_mape = mape(y_test, predict_xgboost)
MAPE.append({'MAPE xgboost': xgboost_mape})
print(f"Точность модели xgboost по метрике MAPE: {(xgboost_mape)*100:0.2f}%")

#### 12.5. GradientBoosting с подбором параметров

Пропишем функцию подбора параметров:

In [181]:
def regularise(X_train, y_train):
    max_depth = [5, 10, 15]
    n_estimators = [100, 200, 1000]
    hyperparameters = dict(max_depth=max_depth, n_estimators=n_estimators)
    model = GradientBoostingRegressor()
    model.fit(X_train, y_train)
    clf = GridSearchCV(model, hyperparameters)
    best_model = clf.fit(X_train, y_train)
    best_max_depth = best_model.best_estimator_.get_params()['max_depth']
    best_n_estimators = best_model.best_estimator_.get_params()['n_estimators']
    return best_max_depth, best_n_estimators

Проводим обучение:

In [182]:
gb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=5, n_estimators=1000)
gb.fit(X_train, np.log(y_train+1))

In [183]:
predict_gb = np.exp(gb.predict(X_test))

Оцениваем точность:

In [184]:
gb_mape = mape(y_test, predict_gb)
MAPE.append({'MAPE GradientBoosting': gb_mape})
print(f"Точность модели Gradient Boosting по метрике MAPE: {(gb_mape)*100:0.2f}%")

#### 12.6. Stacking

Пропишем функцию:

In [185]:
scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test) 
y_train = y_train 
y_test = y_test
cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

def compute_meta_feature(regr, X_train, X_test, y_train, cv):
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)    
    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        folded_regr = clone(regr)
        folded_regr.fit(X_fold_train, y_fold_train)
        X_meta_train[predict_fold_index] = folded_regr.predict(X_fold_predict)
    meta_regr = clone(regr)
    meta_regr.fit(X_train, y_train)
    X_meta_test = meta_regr.predict(X_test)
    return X_meta_train, X_meta_test

def generate_meta_features(regr, X_train, X_test, y_train, cv):
    features = [compute_meta_feature(regr, X_train, X_test, y_train, cv) for regr in tqdm(regr)]    
    stacked_features_train = np.vstack([features_train for features_train, features_test in features]).T
    stacked_features_test = np.vstack([features_test for features_train, features_test in features]).T
    return stacked_features_train, stacked_features_test

X_train = np.where(np.isnan(X_train), 0, X_train)
X_test = np.where(np.isnan(X_test), 0, X_test)
y_train = np.where(np.isnan(y_train), 0, y_train)

Проводим обучение:

In [186]:
regr = RandomForestRegressor(n_estimators=300, min_samples_split=2, min_samples_leaf=1, 
                             max_features=3, max_depth=19, bootstrap=True, random_state=RANDOM_SEED)

stacked_features_train, stacked_features_test = generate_meta_features([
                            regr,
                            GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=10, n_estimators=300),
                            KNeighborsRegressor(n_neighbors=2, algorithm = 'ball_tree', weights = 'distance', p=1),
                            RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1, max_depth=5, n_estimators=200),
                            ExtraTreesRegressor(random_state=RANDOM_SEED), 
                            RandomForestRegressor(random_state=RANDOM_SEED, max_depth=15)], X_train, X_test, y_train, cv)

Оцениваем точность:

In [187]:
def compute_metric(regr, X_train, y_train, X_test, y_test): 
    regr.fit(X_train, y_train) 
    y_test_pred = regr.predict(X_test) 
    return np.round(mape(y_test, y_test_pred)*100, 2)

In [188]:
MAPE_Stacking = compute_metric(regr, stacked_features_train, y_train, stacked_features_test, y_test)

In [189]:
MAPE.append({'MAPE Stacking': MAPE_Stacking})

In [190]:
print(f"Точность модели по метрике MAPE: {MAPE_Stacking}%")

### 13. Сравним точность и выберем модель обучения:

In [191]:
MAPE

### Выводы

- CatBoost и GradientBoosting показывают хороший результат;
- Лучший результат показал xgboost 16.18;
- Результат заметно улучшается после логарифмирования целевой переменной.

### Submission

Делаем предсказания по выбранной модели обучения: 

In [192]:
predict_test = np.exp(xb.predict(X_test))
predict_submission = np.exp(xb.predict(test_data))

Запишем наш вариант решения в требуемый датасет:

In [193]:
sample_submission['price'] = predict_submission
submission = sample_submission.copy()
submission

Запишем в submission в файл (Если нужно):

In [194]:
submission.to_csv('submission.csv', index=False)